# Taxonomy Loader (csv-2-igc-rest)
Copyright : IBM, 2018<br>
Author : **Karl Hegarty**, IBM Ireland<br>

# Industry Models Additional Utilities and Samples
```
Important Note: the items in this project are being shared on an "as-is" basis. Users may copy and modify Source Components and Sample Materials for internal use only provided however that Licensee may not alter or delete any copyright information or notices contained in the Source Components or Sample Materials. IBM provides the Source Components and Sample Materials without obligation of support and "AS IS", WITH NO WARRANTY OF ANY KIND, EITHER EXPRESS OR IMPLIED, INCLUDING THE WARRANTY OF TITLE, NON-INFRINGEMENT OR NON-INTERFERENCE AND THE IMPLIED WARRANTIES AND CONDITIONS OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE.

There is no direct support offered for these components.
IBM, Industry Models Development Team.
```

## Purpose and Notes
Purpose : The following is IGC Loader module which will take a csv format
and using the igc rest API for each row;
 - create a category or term asset
 - create an IsATypeOf relationship (if required)
 - create IsOf, Related Terms and labels
  
Notes:
 - if the glossary is empty 2 categories (Industry Models & Supportive Content) & 1 term (Supportive Content) will be created
 - if these 2 categories and 1 term already exist they will be reused

Please specify the IGC Credentials and url for the values <font color=red>**s.auth**</font>  & <font color=red>**urlprefix**</font> that follow.

In [ ]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json

import os
currdir=os.getcwd()

if "notex" in currdir:
    dirpath, rest = currdir.split("notex",1)
workdir = dirpath + 'notex/'
print ("Current working dir :\t" + workdir + "\n")

### taxonomy for loading 
taxonomy_csv='../extractors/iso20022/output/ISO20022-v2013.csv'

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
s = requests.Session()

###### target IGC instance
s.auth = ('isadmin', '___password_goes_here_____')
urlprefix="https://____hostname____/ibm/iis/igc-rest/v1"         # ISO 20022 test server

r = s.get(urlprefix + "/metadata", verify=False)
sessionidcookie = {'JSESSIONID': r.cookies['JSESSIONID']}
r1 = s.get(urlprefix + "/search/?types=category&text=Supportive%20Content", cookies=sessionidcookie, verify=False)
print (str(r1.status_code) + "\tSuccess : Connected to " + urlprefix)
print (sessionidcookie)

In [ ]:
# Verify CSV / Taxonomy exists
import time
stat=os.stat(taxonomy_csv)
(mode, ino, dev, nlink, uid, gid, size, atime, mtime, ctime) = os.stat(taxonomy_csv)
print(taxonomy_csv + "\tlast modified: %s" % time.ctime(mtime))

In [ ]:
# Import the CSV - first row will be used as headers for dictionary

import csv
import json

def createdictfromcsv(csvfilename, csvindexref):
    with open(csvfilename, 'r', newline='', encoding='utf8') as CSVFile:
        reader = csv.reader(CSVFile)
        csvfile = list(reader)
    for i in range(len(csvfile)):
        if i > 0:
            csvl[i]={}
            for j in range(len(csvfile[i])):
                csvl[i].setdefault(csvfile[0][j].replace('\ufeff',''), csvfile[i][j])

csvl={}
createdictfromcsv(taxonomy_csv, 0)
print ("The csv load file has " + str(len(csvl)) + " row(s)")
print ('fin')

In [ ]:
# check for base cagtegories - reuse if exist, create if they don't
# check for and create if missing the labels required

def ridfromurl(url):
    rtrunc=len(url) - url.rfind("/") -1
    return url[-rtrunc:]

im_cat_rid=""
sc_cat_rid=""
sc_term_rid=""

## check for Industry Models category
headers = { "Accept": "application/json", "Accept-Encoding": "identity" }
bodytext = {
  "properties" : [ "name" ],
  "types" : [ "category"],
  "where" : {
    "conditions" : [{
      "value" : "Industry Models",
      "property" : "name",
      "operator" : "like {0}"
    } ],
    "operator" : "and"
  }
}

r1 = s.post(urlprefix + '/search', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
p1 = json.loads(r1.text)
if r1.status_code != 200 or len(p1['items']) != 1: ## if Industry Models category not found create it
    # create category 'Industry Models'
    catbody = {
          "_type" : "category",
          "short_description" : "Regulatory Platform root category",
          "name" : "Industry Models"
    }
    r1 = s.post(urlprefix + "/assets", cookies=sessionidcookie, verify=False, json=catbody, stream=True)
    if r1.status_code != 201:
        print ("Error - can't create Industry Models category : return code : r1 : " + str(r1.status_code))
        print (r1.headers)
    else:
        im_cat_rid=ridfromurl(r1.headers['Location'])
else:
    for name in p1['items']:
        im_cat_rid=name.get('_id')

        
## check, create for Supportive category
bodytext = {
  "properties" : [ "name" ],
  "types" : [ "category"],
  "where" : {
    "conditions" : [{
      "value" : "Supportive Content",
      "property" : "name",
      "operator" : "like {0}"
    } ],
    "operator" : "and"
  }
}

r1 = s.post(urlprefix + '/search', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
p1 = json.loads(r1.text)
if r1.status_code != 200 or len(p1['items']) != 1:
    # create category 'Supportive Content'
    catbody = {
          "_type" : "category",
          "short_description" : "Regulatory Platform Supportive Content category",
          "parent_category" : im_cat_rid,
          "name" : "Supportive Content"
    }
    r1 = s.post(urlprefix + "/assets", cookies=sessionidcookie, verify=False, json=catbody, stream=True)    
    if r1.status_code != 201:
        print ("Error - can't create Supportive Content category : return code : r1 : " + str(r1.status_code))
        print (r1.headers)
    else:
        sc_cat_rid=ridfromurl(r1.headers['Location'])
else:
    for name in p1['items']:
        sc_cat_rid=name.get('_id')

## check, create for Support Content term
bodytext = {
  "properties" : [ "name" ],
  "types" : [ "term"],
  "where" : {
    "conditions" : [{
      "value" : "Supportive Content",
      "property" : "name",
      "operator" : "like {0}"
    } ],
    "operator" : "and"
  }
}

r1 = s.post(urlprefix + '/search', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
p1 = json.loads(r1.text)
if r1.status_code != 200 or len(p1['items']) != 1:
    # create term 'Supportive Content'
    termbody = {
      "_type" : "term",
      "short_description" : "Regulatory Platform Supportive Content Term",
      "status" : "CANDIDATE",
      "parent_category" : sc_cat_rid,        
      "name" : "Supportive Content"
    }
    r1 = s.post(urlprefix + "/assets", cookies=sessionidcookie, verify=False, json=termbody, stream=True)    
    if r1.status_code != 201:
        print ("Error - can't create Supportive Content Term : return code : r1 : " + str(r1.status_code))
        print (r1.headers)
    else:
        sc_term_rid=ridfromurl(r1.headers['Location'])
else:
    for name in p1['items']:
        sc_term_rid=name.get('_id')

print ("Industry Models Category : \t" + "\t" + im_cat_rid)
print ("Supportive Content Category : \t" + "\t" + sc_cat_rid)
print ("Supportive Content Term : \t" + "\t" + sc_term_rid)

# Create missing labels

labellist={}
labelhash={}
headers = { "Accept": "application/json", "Accept-Encoding": "identity" }
if 'label' in csvl[1]:
    for i in sorted(csvl.keys()):
        if len(csvl[i]['label']) > 0:
            labelhash[i]={}
            label_str=csvl[i]['label'].replace("|", ", ")
            for ilab in label_str.split(', '):             # isATypeOf values can be an array comma + space separated
                if ilab not in labellist:                  # new label encountered
                    labelbody={
                            "properties": ["name"],
                            "types": ["label"],
                            "where": {
                                "conditions": [{
                                    "value": ilab,
                                    "property": "name",
                                    "operator": "like {0}"
                                }],
                                "operator": "and"
                            }
                        }
                    r1 = s.post(urlprefix + '/search', cookies=sessionidcookie, verify=False, json=labelbody, headers=headers)
                    p1 = json.loads(r1.text)
                    if r1.status_code != 200 or len(p1['items']) != 1:         # new label not in catalog
                        print ("didn't find " + ilab + "\n" + str(labelbody))  # create label ilab
                        labelbody = {
                              "_type" : "label",
                              "name" : ilab
                        }
                        r1 = s.post(urlprefix + "/assets", cookies=sessionidcookie, verify=False, json=labelbody, stream=True)
                        if r1.status_code != 201:
                            print ("Error - can't create Industry Models label : return code : r1 : " + str(r1.status_code))
                            print (r1.headers)
                        else:
                            label_rid=ridfromurl(r1.headers['Location'])
                            print ("new label created. Rid = " + ilab + label_rid)
                            labellist.setdefault(ilab, label_rid)
                            labelhash[i][label_rid]={}
                    else:                                                      # get the existing catalog label rid
                        label_rid=ridfromurl(p1['items'][0]['_url'])
                        print ("found " + ilab + "\t" + label_rid)
                        labellist.setdefault(ilab, ridfromurl(p1['items'][0]['_url']))
                        labelhash[i][label_rid]={}
                else:                   
                    label_rid=labellist[ilab]
                    labelhash[i][label_rid]={}
print ('fin')

## API - Part 1 - create all the category and terms assets and store returned rids

In [ ]:
# API #1 create all the category and terms assets and store returned rids

headers = { "Accept": "application/json", "Accept-Encoding": "identity"}
ridhash={}
assethash={}

for i in sorted(csvl.keys()):
    ridhash[i]={}
    if 'parent_category' in csvl[i] and csvl[i]['parent_category']:
        pcat=ridhash[int(csvl[i]['parent_category'])]['rid']
    else:
        pcat=sc_cat_rid
    asset_key=csvl[i]['element_type'] + pcat + csvl[i]['element_name'][:255]
    if asset_key in assethash:
        ridhash[i].setdefault('norid', assethash[asset_key]['index'])
        print ('Duplicate found\t' + asset_key)
    else:
        if 'short_description' in csvl[i]:
            short_description=csvl[i]['short_description'][:255]
        else:
            short_description=''
        if csvl[i]['element_type'] == 'category':
            termbody = {
              "_type" : "category",
              "short_description" : short_description,
              "parent_category" : pcat,
              "name" : csvl[i]['element_name'][:255]              # element name
            }
            if 'custom_External_Mapping' in csvl[i] and len(csvl[i]['custom_External_Mapping']) > 0:
                termbody["custom_External_Mapping"] = csvl[i]['custom_External_Mapping']  
        else:
            termbody = {
              "_type" : "term",
              "short_description" : csvl[i]['short_description'][:255],
              "status" : "CANDIDATE",
              "parent_category" : pcat,
              "name" : csvl[i]['element_name'][:255]               # element name
            }
            if len(csvl[i]['long_description']) > 0:
                termbody["long_description"] = csvl[i]['long_description']
            if 'example' in csvl[i] and len(csvl[i]['example']) >0:
                termbody["example"] = csvl[i]['example']                  
            if 'external_reference' in csvl[i] and len(csvl[i]['external_reference']) > 0:
                termbody["custom_Formula"] = csvl[i]['external_reference']
            if 'custom_external_reference' in csvl[i] and len(csvl[i]['custom_external_reference']) > 0:
                termbody["custom_External Document Reference"] = csvl[i]['custom_external_reference']
            if 'custom_External_Mapping' in csvl[i] and len(csvl[i]['custom_External_Mapping']) > 0:
                termbody["custom_External_Mapping"] = csvl[i]['custom_External_Mapping']                
            if 'custom_Cardinality' in csvl[i] and len(csvl[i]['custom_Cardinality']) > 0:
                termbody["custom_Cardinality"] = csvl[i]['custom_Cardinality']
            if 'custom_Data Type' in csvl[i] and len(csvl[i]['custom_Data Type']) > 0:
                termbody["custom_Data Type"] = csvl[i]['custom_Data Type']  
            if 'custom_Enumeration_Values' in csvl[i] and len(csvl[i]['custom_Enumeration_Values']) > 0:
                termbody["custom_Enumeration_Values"] = csvl[i]['custom_Enumeration_Values']                  
        termjson=json.dumps(termbody)
        r1 = s.post(urlprefix + "/assets", cookies=sessionidcookie, verify=False, json=termbody, stream=True)
        if r1.status_code != 201:
            print ("return code : r1 : " + str(r1.status_code) + "\t: row = " + str(i) + "\tname =" + csvl[i]['element_name'])
            print (json.dumps(termbody, sort_keys=True, indent=4, separators=(',', ': ')))
        else:
            assethash[asset_key]={}
            returnurl = r1.headers['Location']
            ridval=ridfromurl(returnurl)
            ridhash[i].setdefault('rid', ridval)
            assethash[asset_key].setdefault('index', i)
print ('fin')

## API - Part 2 - add is_a_type_of properties

In [ ]:
# API #2 add is_a_type_of properties
for i in sorted(csvl.keys()):
    bodyprefix="{\n\t\"items\": [\n"
    bodysuffix="\t],\n\t\"mode\":\"replace\"\n}"
    if i > 0 and 'rid' in ridhash[i] and csvl[i]['element_type'] == 'term' and 'isATypeOf' in csvl[i] and len(csvl[i]['isATypeOf']) > 0:
        isatypeof_str=csvl[i]['isATypeOf']
        idd={}
        for iss in isatypeof_str.split(', '):             # IsATypeOf values can be an array comma + space separated
            if int(iss) in ridhash:
                if 'rid' not in ridhash[int(iss)]:
                    ridindex=ridhash[int(iss)]['norid']
                    rid=ridhash[ridindex]['rid']
                else:
                    rid=ridhash[int(iss)]['rid']
                if rid not in idd:
                    #print (str(i) + "   \"_id\": \"" + ridhash[int(iss)]['rid'] + "\"")
                    idd.setdefault(rid)
        if len(idd) == 1:
            isatypeofbody=bodyprefix
            isatypeofbody=isatypeofbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[0] + "\"\n\t}\n"
            isatypeofbody=isatypeofbody + bodysuffix            
        else:
            for ids in range(len(idd)):
                if ids == 0:
                    isatypeofbody=bodyprefix
                    isatypeofbody=isatypeofbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t},\n"
                elif ids == len(idd) - 1:
                    isatypeofbody=isatypeofbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t}\n"    # no ending comma
                else:
                    isatypeofbody=isatypeofbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t},\n"
            isatypeofbody=isatypeofbody + bodysuffix
#        print (isatypeofbody)
        isatypeofjson=json.loads(isatypeofbody)
        r2 = s.put(urlprefix + "/assets/" +  ridhash[i]['rid'] + "/is_a_type_of/", cookies=sessionidcookie, verify=False, json=isatypeofjson, headers=headers, stream=True)
        if r2.status_code != 200:
            print ("rid = \t" + ridhash[i]['rid'])
            print ("error adding into term hierarcy: " + str(r2.status_code) + " " + str(i) + ":\tname = " + csvl[i]['element_name'])
            print (isatypeofbody)
    elif i > 0 and 'rid' in ridhash[i] and csvl[i]['element_type'] == 'term':
        isatypeofbody=bodyprefix  + "\t{\n\t\"_id\": \"" + sc_term_rid + "\"\n\t}\n" + bodysuffix
        isatypeofjson=json.loads(isatypeofbody)
        r2 = s.put(urlprefix + "/assets/" +  ridhash[i]['rid'] + "/is_a_type_of/", cookies=sessionidcookie, verify=False, json=isatypeofjson, headers=headers, stream=True)
        if r2.status_code != 200:
            print ("rid = \t" + ridhash[i]['rid'])
            print ("error adding into  hierarcy / supportive content term: " + str(r2.status_code) + " " + str(i) + ":\tname = " + csvl[i]['element_name'])
            print (isatypeofbody)

print ('fin')

## API - Part 3 - add related terms properties

In [ ]:
# API #3 add related terms properties
for i in sorted(csvl.keys()):
    bodyprefix="{\n\t\"items\": [\n"
    bodysuffix="\t],\n\t\"mode\":\"replace\"\n}"
    if i > 0 and 'rid' in ridhash[i] and csvl[i]['element_type'] == 'term' and 'RelatedTerms' in csvl[i] and len(csvl[i]['RelatedTerms']) > 0:
        related_terms_str=csvl[i]['RelatedTerms']
        idd={}
        for irt in related_terms_str.split(', '):             # RelatedTerms values can be an array comma + space separated
            if int(irt) in ridhash:
                if 'rid' not in ridhash[int(irt)]:            # if no 'rid' then it referes to a duplicate
                    ridindex=ridhash[int(irt)]['norid']
                    rid=ridhash[ridindex]['rid']
                else:
                    rid=ridhash[int(irt)]['rid']
                if rid not in idd:
                    #print (str(i) + "   \"_id\": \"" + ridhash[int(iss)]['rid'] + "\"")
                    idd.setdefault(rid)
        if len(idd) == 1:                                     # only a single Related Term
            relatedtermbody=bodyprefix
            relatedtermbody=relatedtermbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[0] + "\"\n\t}\n"
            relatedtermbody=relatedtermbody + bodysuffix            
        else:
            relatedtermbody=''
            for ids in range(len(idd)):                       # multiple related terms
                if ids == 0:
                    relatedtermbody=bodyprefix + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t},\n"
                elif ids == len(idd) - 1:
                    relatedtermbody=relatedtermbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t}\n"    # no ending comma
                else:
                    relatedtermbody=relatedtermbody + "\t{\n\t\"_id\": \"" + list(idd.keys())[ids] + "\"\n\t},\n"
            relatedtermbody=relatedtermbody + bodysuffix
        relatedtermjson=json.loads(relatedtermbody)
        r2 = s.put(urlprefix + "/assets/" +  ridhash[i]['rid'] + "/related_terms/", cookies=sessionidcookie, verify=False, json=relatedtermjson, headers=headers, stream=True)
        if r2.status_code != 200:
            print ("rid = \t" + ridhash[i]['rid'])
            print ("error adding RelatedTerm: " + str(r2.status_code) + " " + str(i) + ":\tname = " + csvl[i]['element_name'])
            print (relatedtermjson)
print ('fin')

## API - Part 4 - add is_of properties

In [ ]:
# API #4 add is_of properties
for i in sorted(csvl.keys()):
    bodyprefix="{\n\t\"items\": [\n"
    bodysuffix="\t],\n\t\"mode\":\"replace\"\n}"
    if i > 0 and 'IsOf' in csvl[i] and len(csvl[i]['IsOf']) > 0:
        isof_val=csvl[i]['IsOf']
        isof_rid=ridhash[int(isof_val)]['rid']
        isofbody=bodyprefix +  "\t{\n\t\"_id\": \"" + str(isof_rid) + "\"\n\t}\n" + bodysuffix
        isofjson=json.loads(isofbody)
        r2 = s.put(urlprefix + "/assets/" +  ridhash[i]['rid'] + "/is_of/", cookies=sessionidcookie, verify=False, json=isofjson, headers=headers, stream=True)
        if r2.status_code != 200:
            print ("rid = \t" + ridhash[i]['rid'])
            print ("error adding into Is_Of : " + str(r2.status_code) + " " + str(i) + ":\tname = " + csvl[i]['IsOf'] + "\t" + rid)
            print (relatedtermjson)
print ('fin')

## API - Part 5 - add labels

In [ ]:
# API # 5 Add labels
labelprefix="{\n\t\"items\": [\n"
labelsuffix="\t],\n\t\"mode\":\"replace\"\n}"
for i in labelhash.keys(): 
    rid_val=ridhash[i]['rid']
    if len(labelhash[i]) == 1:
        labelbody=labelprefix + "\t{\n\t\"_id\": \"" + list(labelhash[i].keys())[0] + "\"\n\t}\n"
        labelbody=labelbody + labelsuffix
    else:
        for ids in range(len(labelhash[i])):                       # multiple related terms
            if ids == 0:
                labelbody=labelprefix + "\t{\n\t\"_id\": \"" + list(labelhash[i].keys())[ids] + "\"\n\t},\n"
            elif ids == len(labelhash[i]) - 1:
                labelbody=labelbody + "\t{\n\t\"_id\": \"" + list(labelhash[i].keys())[ids] + "\"\n\t}\n"    # no ending comma
                labelbody=labelbody + labelsuffix
            else:
                labelbody=labelbody + "\t{\n\t\"_id\": \"" + list(labelhash[i].keys())[ids] + "\"\n\t},\n"
    #print(labelbody)
    labeljson=json.loads(labelbody)
    r2 = s.put(urlprefix + "/assets/" +  rid_val + "/labels/", cookies=sessionidcookie, verify=False, json=labeljson, headers=headers, stream=True)
    if r2.status_code != 200:
        print ("rid = \t" + rid_val)
        print ("error adding label : " + str(r2.status_code) + " " + str(i) + ":\trid = " + rid_val + "\t" )
        print (labeljson)
print ('fin')

## debug and other useful info

In [ ]:
debug=True
debug=False
if debug:
    ## Summary of taxonomy load

    bodytext = {
      "pageSize": "10000",      
      "types" : ["term"],
      "properties" : ["name", "type" ],    
      "where" : {
        "conditions" : [{
          "value" : "A. Applicability of GAAP",
          "property" : "name",
          "operator" : "like {0}"
        }],
        "operator" : "and"
      }
    }

    r1 = s.get(urlprefix + '/search/category', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
    r2 = s.get(urlprefix + '/search/term', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
    if r1.status_code != 200:
        print("r1.error\t" + str(r1.status_code))
    if r2.status_code != 200:
        print("r2.error\t" + str(r2.status_code))    

    p1 = json.loads(r1.text)
    p2 = json.loads(r2.text)

    print ("category total\t=\t" + str(p1['paging']['numTotal']))
    print ("term total\t=\t"  + str(p2['paging']['numTotal']))
else:
    print ("debug mode : Off")

In [ ]:
if debug:
    headers = { "Accept": "application/json", "Accept-Encoding": "identity" }
    bodytext = {
      "properties" : [ "_name" ],
      "types" : [ "custom_attribute"],
      "where" : {
        "conditions" : [{
          "value" : "custom_External_Mapping",
          "property" : "custom_attribute._name",
          "operator" : "like {0}"
        } ],
        "operator" : "and"
      }
    }

    r1 = s.get(urlprefix + '/search/custom_attribute', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
    p1 = json.loads(r1.text)

    if r1.status_code != 200:
        print ("Error - can't find custom_External_Mapping : return code : r1 : " + str(r1.status_code))
        print (r1.headers)
    else:
        for i in p1['items']:
            testname=i.get('_name')
            print (testname)
    #        print (json.dumps(p1['items'], sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
if debug:
    headers = { "Accept": "application/json", "Accept-Encoding": "identity" }
    bodytext = {
      "types" : [ "custom_attribute"],
      "where" : {
        "conditions" : [{
          "value" : "External_Mapping",
          "property" : "name",
          "operator" : "like {0}"
        } ],
        "operator" : "and"
      }
    }

    r1 = s.post(urlprefix + '/search', cookies=sessionidcookie, verify=False, json=bodytext, headers=headers)
    p1 = json.loads(r1.text)
    if r1.status_code != 200:
        print ("Error - can't find custom_External_Mapping : return code : r1 : " + str(r1.status_code))
        print (r1.headers)
    else:
        for i in p1['items']:
            testname=i.get('_name')
            print (testname)
    #        print (json.dumps(p1['items'], sort_keys=True, indent=4, separators=(',', ': ')))

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')